<a href="https://colab.research.google.com/github/JoshDumo/QisNatureTut/blob/main/QN_ElectronicStructure_HH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electronic Structure

Please see the tutorial at
https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html
and watch the videos from the summer school
https://www.youtube.com/watch?v=2XEjrwWhr88

Here, let's focus on making this work in the Google Colab.

## Installing Qiskit and Qiskit Nature
PySCF is also needed for classical Hartree-Fock calculation. Check it out at https://pyscf.org/index.html


In [1]:
!pip install qiskit
!pip install qiskit_nature

     |████████████████████████████████| 6.0MB 8.6MB/s 
     |████████████████████████████████| 17.9MB 210kB/s 
     |████████████████████████████████| 204kB 36.6MB/s 
     |████████████████████████████████| 215kB 41.3MB/s 
     |████████████████████████████████| 2.1MB 32.3MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 1.0MB 20.9MB/s 
     |████████████████████████████████| 194kB 41.0MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 614kB 40.8MB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
     |████████████████████████████████| 5.5MB 42.5MB/s 
  Created wheel for qiskit: filename=qiskit-0.25.2-cp37-none-any.whl size=3066 sha256=655a5e5f336a7695260fdfca21d34af367d4c85872d720a2ea902a4b6fc8ebbc
  Stored in directory: /root/.cache/pip/wheels/47/a6/95/c06fb36fa3f4de8eb325c737eb8164c1805314f9f5fc941e5b
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

In [2]:
!pip install pyscf

     |████████████████████████████████| 29.7MB 147kB/s 


Now we can follow the rest of the tutorial
### Setting up a H molecule and the driver

In [3]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')

### Setting up the problem and a Second Quantized Operator

In [4]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

In [5]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

  +-+- * (0.18093119978423158+0j)
+ +--+ * (-0.18093119978423156+0j)
+ -++- * (-0.18093119978423156+0j)
+ -+-+ * (0.18093119978423156+0j)
+ IIIN * (-0.47189600728114073+0j)
+ IINI * (-1.2563390730032507+0j)
+ IINN * (0.48365053047106565+0j)
+ INII * (-0.47189600728114073+0j)
+ ININ * (0.6985737227320187+0j)
+ INNI * (0.6645817302552972+0j)
+ NIII * (-1.2563390730032507+0j)
+ NIIN * (0.6645817302552972+0j)
+ NINI * (0.675710154803517+0j)
+ NNII * (0.48365053047106565+0j)


### Transform the Hamiltonian to get the Qubit operator
Note the use of the Jordan-Wigner transformation.

In [6]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

-0.8105479805373268 * IIII
- 0.22575349222402502 * ZIII
+ 0.17218393261915543 * IZII
+ 0.12091263261776641 * ZZII
- 0.22575349222402505 * IIZI
+ 0.17464343068300467 * ZIZI
+ 0.1661454325638243 * IZZI
+ 0.17218393261915546 * IIIZ
+ 0.1661454325638243 * ZIIZ
+ 0.16892753870087926 * IZIZ
+ 0.12091263261776641 * IIZZ
+ 0.04523279994605789 * XXXX
+ 0.04523279994605789 * YYXX
+ 0.04523279994605789 * XXYY
+ 0.04523279994605789 * YYYY


Now Parity Mapping

In [7]:
qubit_converter = QubitConverter(mapper = ParityMapper(), two_qubit_reduction=True)
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
print(qubit_op)

-1.0523732457728592 * II
+ (-0.3979374248431804+1.3877787807814457e-17j) * ZI
+ 0.3979374248431804 * IZ
- 0.011280104256235296 * ZZ
+ (0.18093119978423153+3.469446951953614e-18j) * XX
